# Predicción de valor de coches usados

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Descripcion de los datos

__Características__

* __`DateCrawled`__ — fecha en la que se descargó el perfil de la base de datos


* __`VehicleType`__ — tipo de carrocería del vehículo


* __`RegistrationYear`__ — año de matriculación del vehículo


* __`Gearbox`__ — tipo de caja de cambios


* __`Power`__ — potencia (CV)


* __`Model`__ — modelo del vehículo


* __`Mileage`__ — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)


* __`RegistrationMonth`__ — mes de matriculación del vehículo


* __`FuelType`__ — tipo de combustible


* __`Brand`__ — marca del vehículo


* __`NotRepaired`__ — vehículo con o sin reparación


* __`DateCreated`__ — fecha de creación del perfil


* __`NumberOfPictures`__ — número de fotos del vehículo


* __`PostalCode`__ — código postal del propietario del perfil (usuario)


* __`LastSeen`__ — fecha de la última vez que el usuario estuvo activo

__Objetivo__

* __`Price`__ — precio (en euros)

## Preparación de datos

### Carga de Datos

In [1]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import time

In [2]:
# Cargar la base de datos y converirta en DataFrame
df = pd.read_csv('/datasets/car_data.csv')

df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


Vamos a normalizar las columnas para que sus nombres sean más descriptivos, consistentes y fáciles de entender, lo que facilitará el análisis y la interpretación de los datos.

In [3]:
# Normalización de columnas

# Diccionario con los nuevos nombres
new_column_names = {
    'DateCrawled': 'date_crawled',
    'Price': 'price',
    'VehicleType': 'vehicle_type',
    'RegistrationYear': 'registration_year',
    'Gearbox': 'gearbox',
    'Power': 'power',
    'Model': 'model',
    'Mileage': 'mileage',
    'RegistrationMonth': 'registration_month',
    'FuelType': 'fuel_type',
    'Brand': 'brand',
    'NotRepaired': 'not_repaired',
    'DateCreated': 'date_created',
    'NumberOfPictures': 'number_of_pictures',
    'PostalCode': 'postal_code',
    'LastSeen': 'last_seen'
}

# Renombrar las columnas
df.rename(columns=new_column_names, inplace=True)

# Verificar los nuevos nombres de las columnas
print(df.columns)

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'mileage', 'registration_month', 'fuel_type', 'brand',
       'not_repaired', 'date_created', 'number_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


In [4]:
# Verificar el tipo de datos para cada columna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

### Transformación de datos

Eliminamos registros con años fuera de rango (como los valores menores a 1900 o mayores a 2022).

In [5]:
# Eliminar registros con valores anómalos en 'RegistrationYear'
df = df[(df['registration_year'] >= 1900) & (df['registration_year'] <= 2022)]
df

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [6]:
# Verificar existencia de valores nulos
df.isnull().sum()

date_crawled              0
price                     0
vehicle_type          37319
registration_year         0
gearbox               19695
power                     0
model                 19630
mileage                   0
registration_month        0
fuel_type             32767
brand                     0
not_repaired          71007
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

In [7]:
# Calcular la proporción de valores nulos por columna
null_proportion = df.isnull().sum() / len(df)
print(null_proportion)

date_crawled          0.000000
price                 0.000000
vehicle_type          0.105362
registration_year     0.000000
gearbox               0.055604
power                 0.000000
model                 0.055421
mileage               0.000000
registration_month    0.000000
fuel_type             0.092510
brand                 0.000000
not_repaired          0.200473
date_created          0.000000
number_of_pictures    0.000000
postal_code           0.000000
last_seen             0.000000
dtype: float64


En lugar de imputar valores nulos con la moda global, imputaremos basándonos en grupos específicos para reducir el sesgo.

1. Uso de `transform()` en lugar de `apply()`: Esto mantiene la alineación de los resultados con el DataFrame original, lo cual es crucial para una imputación mas precisa.
<br>

2. Manejo de casos sin moda clara: Si la moda no puede ser calculada debido a la falta de datos en un grupo específico, se imputa con un valor predeterminado (como 'unknown', 'manual', o 'other').
<br>

3. Estrategia general:

- `vehicle_type` se imputa según la moda dentro de cada brand.

- `gearbox` se imputa según la moda dentro de cada `vehicle_type`.

- `model` se imputa según la moda dentro de cada `brand`.

- `fuel_type` se imputa según la moda dentro de cada `vehicle_type`.

- `not_repaired` se imputa directamente con `unknown`.

In [8]:
# Imputar 'vehicle_type' basado en la moda dentro de cada 'brand'
df['vehicle_type'] = df.groupby('brand')['vehicle_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))

# Imputar 'gearbox' basado en la moda dentro de cada 'vehicle_type'
df['gearbox'] = df.groupby('vehicle_type')['gearbox'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'manual'))

# Imputar 'model' basado en la moda dentro de cada 'brand'
df['model'] = df.groupby('brand')['model'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'other'))

# Imputar 'fuel_type' basado en la moda dentro de cada 'vehicle_type'
df['fuel_type'] = df.groupby('vehicle_type')['fuel_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))

# Imputar 'not_repaired' con 'unknown'
df['not_repaired'].fillna('unknown', inplace=True)

# Verificar que los valores nulos hayan sido manejados
print(df.isnull().sum())

/tmp/ipykernel_389/88137888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vehicle_type'] = df.groupby('brand')['vehicle_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
/tmp/ipykernel_389/88137888.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gearbox'] = df.groupby('vehicle_type')['gearbox'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'manual'))


date_crawled          0
price                 0
vehicle_type          0
registration_year     0
gearbox               0
power                 0
model                 0
mileage               0
registration_month    0
fuel_type             0
brand                 0
not_repaired          0
date_created          0
number_of_pictures    0
postal_code           0
last_seen             0
dtype: int64


/tmp/ipykernel_389/88137888.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['model'] = df.groupby('brand')['model'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'other'))
/tmp/ipykernel_389/88137888.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fuel_type'] = df.groupby('vehicle_type')['fuel_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithC

Este enfoque debería ser eficaz para reducir el sesgo y manejar los valores nulos de manera contextual, usando la relación entre diferentes variables.

In [9]:
df.describe()

,price,registration_year,power,mileage,registration_month,number_of_pictures,postal_code
count,354198.000000,354198.000000,354198.000000,354198.000000,354198.000000,354198.0,354198.000000
mean,4417.651314,2003.084789,110.078242,128267.607383,5.716819,0.0,50511.793813
std,4514.081022,7.536418,189.536766,37823.538557,3.725539,0.0,25783.464340
min,0.000000,1910.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,2019.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


En general, los datos muestran una gran variabilidad en las características de los vehículos. La distribución de los precios está sesgada hacia la derecha, sugiriendo la presencia de vehículos con precios significativamente altos.

Ahora que los datos están limpios, el siguiente paso es preparar los datos para el entrenamiento de modelos, incluyendo:

* __Codificación de Variables Categóricas__: Codificar las variables categóricas para los modelos que lo requieran (One-Hot Encoding para regresión lineal y Random Forest).
<br>

* __Entrenamiento de Modelos__: Entrenar y evaluar modelos de regresión lineal, árboles de decisión, Random Forest, LightGBM y Catboost.

## Entrenamiento del modelo 

__1. Codificación de Variables Categóricas__:

* Utilizaremos `One-Hot Encoding` (OHE) para las variables categóricas cuando entrenemos modelos que lo requieran, como la `regresión lineal` y `Random Forest`.

* `LightGBM` tiene la capacidad de manejar variables categóricas de manera nativa, por lo que no es necesario aplicar OHE en este caso.

__2. División del Dataset__:

* Dividiremos el dataset en conjuntos de entrenamiento y prueba.

__3. Entrenamiento de Diferentes Modelos__:

* Comenzaremos con `regresión lineal` como línea base.

* `Árbol de Decisión`.

* `Random Forest`.

* `LightGBM`.

__4. Evaluación de Modelos__:

* Usaremos la métrica `RMSE` para evaluar la calidad de las predicciones.

* Compararemos el tiempo de entrenamiento y la velocidad de predicción.

In [10]:
# Eliminar las columnas originales de tiempo
df = df.drop(['date_crawled', 'date_created', 'last_seen'], axis=1)

# Aplicar One-Hot Encoding a las características categóricas restantes
df_ohe = pd.get_dummies(df, drop_first=True)

In [11]:
# Separar características y variable objetivo
X = df_ohe.drop(['price'], axis=1)
y = df_ohe['price']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12345)

## Análisis del modelo

### Regresión Lineal

Entrenamos un modelo de regresión lineal para establecer una línea base de comparación.

In [12]:
# Modelo de regresión lineal
start_time = time.time()
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
pred_lin_reg = lin_reg.predict(X_test)
rmse_lin_reg = np.sqrt(mean_squared_error(y_test, pred_lin_reg))
lin_reg_time = time.time() - start_time
print(f'Linear Regression RMSE: {rmse_lin_reg}, Time: {lin_reg_time:.2f} seconds')

Linear Regression RMSE: 2984.860684759657, Time: 11.54 seconds


### Arbol de Desición

Entrenamos un modelo de Árbol de Decisión y evaluamos su desempeño.

In [13]:
# Modelo de árbol de decisión
start_time = time.time()
tree = DecisionTreeRegressor(random_state=12345)
tree.fit(X_train, y_train)
pred_tree = tree.predict(X_test)
rmse_tree = np.sqrt(mean_squared_error(y_test, pred_tree))
tree_time = time.time() - start_time
print(f'Decision Tree RMSE: {rmse_tree}, Time: {tree_time:.2f} seconds')

Decision Tree RMSE: 2260.5013120670133, Time: 6.72 seconds


###  Random Forest

Entrenamos un modelo de Random Forest con 100 estimadores y evaluamos su desempeño.

In [14]:
# Modelo de Random Forest
start_time = time.time()
forest = RandomForestRegressor(n_estimators=10, random_state=12345)
forest.fit(X_train, y_train)
pred_forest = forest.predict(X_test)
rmse_forest = np.sqrt(mean_squared_error(y_test, pred_forest))
forest_time = time.time() - start_time
print(f'Random Forest RMSE: {rmse_forest}, Time: {forest_time:.2f} seconds')

Random Forest RMSE: 1783.9464944328736, Time: 40.80 seconds


### LightGBM

Entrenamos un modelo de LightGBM y evaluamos su desempeño

In [15]:
# Modelo de LightGBM
start_time = time.time()
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'seed': 12345
}

model_lgb = lgb.train(params, lgb_train, valid_sets=lgb_valid, early_stopping_rounds=10)
pred_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
rmse_lgb = np.sqrt(mean_squared_error(y_test, pred_lgb))
lgb_time = time.time() - start_time
print(f'LightGBM RMSE: {rmse_lgb}, Time: {lgb_time:.2f} seconds')

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007895 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1194
[LightGBM] [Info] Number of data points in the train set: 283358, number of used features: 293
[LightGBM] [Info] Start training from score 4422.750905
[1]	valid_0's rmse: 4198.29
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 3929.74
[3]	valid_0's rmse: 3695.34
[4]	valid_0's rmse: 3491.48
[5]	valid_0's rmse: 3315.18
[6]	valid_0's rmse: 3160.71
[7]	valid_0's rmse: 3026.73
[8]	valid_0's rmse: 2909.36
[9]	valid_0's rmse: 2805.1
[10]	valid_0's rmse: 2716.84
[11]	valid_0's rmse: 2634.79
[12]	valid_0's rmse: 2562.24
[13]	valid_0's rmse: 2502.07
[14]	valid_0's rmse: 2446.17
[15]	valid_0's rmse: 2399.31
[16]	valid_0's rmse: 2355.1
[17]	valid_0's rmse: 2318.78
[18]	valid_0's rmse: 2285.3
[19]	valid_0's 

### CatBoost

In [16]:
from catboost import CatBoostRegressor

# Modelo de CatBoost
start_time = time.time()
catboost_model = CatBoostRegressor(iterations=150, learning_rate=0.1, depth=6, random_seed=12345, verbose=0)
catboost_model.fit(X_train, y_train)
pred_catboost = catboost_model.predict(X_test)
rmse_catboost = np.sqrt(mean_squared_error(y_test, pred_catboost))
catboost_time = time.time() - start_time
print(f'CatBoost RMSE: {rmse_catboost}, Time: {catboost_time:.2f} seconds')

CatBoost RMSE: 1920.6059790351956, Time: 5.63 seconds


### XGBoost

In [27]:
import xgboost as xgb

# Preparar los datos para XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definir los parámetros del modelo XGBoost
params = {
    'objective': 'reg:squarederror',  # Tipo de problema: regresión
    'learning_rate': 0.2,
    'max_depth': 4,
    'seed': 12345
}

# Entrenar el modelo XGBoost
start_time = time.time()
xgb_model = xgb.train(params, dtrain, num_boost_round=10)
xgb_time = time.time() - start_time

# Realizar predicciones
pred_xgb = xgb_model.predict(dtest)

# Calcular el RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test, pred_xgb))

print(f'XGBoost RMSE: {rmse_xgb}, Time: {xgb_time:.2f} seconds')

XGBoost RMSE: 2425.8268576118066, Time: 25.41 seconds


# Conclusión

* __Mejor Precisión__: `Random Forest` tiene el `RMSE` más bajo, lo que sugiere que es el modelo más preciso, aunque su tiempo de entrenamiento es significativamente mayor.
<br>

* __Mejor Balance__: `LightGBM` ofrece un excelente equilibrio entre precisión y velocidad, siendo uno de los más rápidos y con un `RMSE` bajo.
<br>

* __Rendimiento Sólido__: `CatBoost` también ofrece un rendimiento sólido, especialmente en conjuntos de datos con muchas variables categóricas.

Para un equilibrio óptimo entre precisión y tiempo de entrenamiento, `LightGBM` sería la `mejor opción`. Si se busca la mayor precisión posible sin importar el tiempo de entrenamiento, `Random Forest` podría ser preferible. Sin embargo, para tareas en las que el manejo de variables categóricas es crucial, `CatBoost` también es una excelente opción.